In [22]:
import mlflow
import os

mlflow.set_tracking_uri("file://" + os.path.expanduser("~/mlruns"))

# remove all existing experiments
experiments = mlflow.search_experiments()
for exp in experiments:
    print(f"{exp.experiment_id} — {exp.name}")

# Create experiment if not exists
if not mlflow.get_experiment_by_name("Default"):
    mlflow.create_experiment("Default")

mlflow.set_experiment("Diabetes")  # Will ensure it uses or creates Default


660696925410320420 — Diabetes
160373978290524396 — Default


<Experiment: artifact_location='file:///home/dhiraj-patra/mlruns/660696925410320420', creation_time=1753104452339, experiment_id='660696925410320420', last_update_time=1753104452339, lifecycle_stage='active', name='Diabetes', tags={}>

In [23]:
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
import pandas as pd

In [24]:
# load data
data = load_diabetes()
column_names = data.feature_names

In [25]:
X_train, x_test, Y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

In [26]:
# start mlflow run
with mlflow.start_run():
    # train model
    n_estimators = 100
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=6, max_features=3)

    # train model
    model.fit(X_train, Y_train)

    # predict model
    predictions = model.predict(x_test)

    input_example = pd.DataFrame(x_test[:1], columns=column_names)
    signature = mlflow.models.infer_signature(x_test, predictions)

    # evaluate model
    mse = mean_squared_error(y_test, predictions)

    # log
    mlflow.log_metric("mse", mse)
    mlflow.sklearn.log_model(
        model,
        name="random_forest_model",
        input_example=input_example,
        signature=signature,
        registered_model_name="sk-learn-random-forest-reg-model"
    )
    print("Logged model with mse: %f" % mse)

Logged model with mse: 2829.249463


Registered model 'sk-learn-random-forest-reg-model' already exists. Creating a new version of this model...
Created version '2' of model 'sk-learn-random-forest-reg-model'.


### now run in terminal and follow the instruction
mlflow ui